# Preprocessing for training

In [ ]:
from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True)

from src.nlp.preprocessing import clean

In [ ]:
from datasets.getters import load_reviews_Review_Label

data = load_reviews_Review_Label()
data

In [ ]:
%%time

data['review'] = data['review'].parallel_apply(clean, args=(TreebankWordTokenizer().tokenize, 
                                                            nltk.corpus.stopwords.words('russian')))
data

In [ ]:
with open('datasets/clean_dataset.df', 'wb') as f:
    dill.dump(data, f)